# Policy Iteration

The following is an implementation of policy iteration exactly as described in [Sutton and Barro’s Introduction to Reinforcement Learning](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf). I have also used [Stanford Universities CS234](https://web.stanford.edu/class/cs234/assignments.html) assignment 1 as a template. Some other resources include [Deep Lizard's RL course](https://deeplizard.com/course/rlcpailzrd) and [Steve Brunton's YouTube playlist on RL](https://www.youtube.com/playlist?list=PLMrJAkhIeNNQe1JXNvaFvURxGY4gE9k74).

### Setup

**Game: [Frozen Lake](https://www.gymlibrary.dev/environments/toy_text/frozen_lake/)**

Winter is here. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend. The surface is described using a grid like the following:

`SFFF`<br>
`FHFH`<br>
`FFFH`<br>
`HFFG`

`S`: starting point, safe  
`F`: frozen surface, safe  
`H`: hole, fall to your doom  
`G`: goal, where the frisbee is located

Your task is to use use *policy iteration* and *value iteration* to come up with an optimal policy for crossing the lake. Frame the task as an MDP with the following reward structure:
- You receive a reward of 1 if you reach the goal, and 0 otherwise.
- The episode ends when you reach the goal or fall in a hole. 

In [1]:
import gym
import numpy as np

from IPython.display import clear_output
from time import sleep

In [2]:
print('Gym version:', gym.__version__)

Gym version: 0.21.0


### Policy Iteration Code

```
For policy_evaluation, policy_improvement, policy_iteration and value_iteration,
the parameters P, nS, nA, gamma are defined as follows:

	P: nested dictionary
		From gym.core.Environment
		For each pair of states in [0, nS-1] and actions in [0, nA-1], P[state][action] is a
		tuple of the form (probability, nextstate, reward, terminal) where
			- probability: float
				the probability of transitioning from "state" to "nextstate" with "action"
			- nextstate: int
				denotes the state we transition to (in range [0, nS - 1])
			- reward: int
				either 0 or 1, the reward for transitioning from "state" to
				"nextstate" with "action"
			- terminal: bool
			  True when "nextstate" is a terminal state (hole or goal), False otherwise
	nS: int
		number of states in the environment
	nA: int
		number of actions in the environment
	gamma: float
		Discount factor. Number in range [0, 1)
```

In [3]:
def policy_evaluation(P, nS, nA, policy, gamma=0.9, tol=1e-3, max_iter=100):
    """
    Evaluate the value function from a given policy.

    Parameters
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    policy: np.array[nS]
        The policy to evaluate. Maps states to actions.
    tol: float
        Terminate policy evaluation when
            max |value_function(s) - prev_value_function(s)| < tol
    Returns
    -------
    value_function: np.ndarray[nS]
        The value function of the given policy, where value_function[s] is
        the value of state s
    """
    
    value_function = np.zeros(nS)
    
    for i in range(max_iter):
        delta = 0
        
        for s in range(nS):
            v = value_function[s] # save a copy of original value of state s
            a = policy[s] # action to take according to policy
            value_function[s] = sum([prob*(reward + gamma*value_function[next_S]) for prob, next_S, reward, term in P[s][a]])
            delta = max(delta, abs(v-value_function[s]))
        
        if delta<tol:
            break
        
        if i==max_iter-1:
            print(f'Policy evaluation failed to converge after {max_iter} iterations')
    
    return value_function

In [4]:
# test (should return reasonable value function, albeit random and usually all zero)
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False)
policy = np.random.randint(low=0, high=env.nA, size=env.nS)
policy_evaluation(env.P, env.nS, env.nA, policy)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [5]:
def policy_improvement(P, nS, nA, value_from_policy, policy, gamma=0.9):
	"""
	Given the value function from policy improve the policy.

	Parameters
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	value_from_policy: np.ndarray
		The value calculated from the policy
	policy: np.array
		The previous policy.

	Returns
	-------
	new_policy: np.ndarray[nS]
		An array of integers. Each integer is the optimal action to take
		in that state according to the environment dynamics and the
		given value function.
	"""

	new_policy = np.zeros(nS, dtype='int')

	policy_stable = True
	for s in range(nS):
		action_values = [sum([prob*(reward + gamma*value_from_policy[next_S]) for prob, next_S, reward, term in P[s][a]]) for a in range(nA)]
		new_policy[s] = np.argmax(action_values)
		if new_policy[s]!=policy[s]:
			policy_stable = False

	return new_policy, policy_stable

In [6]:
# test (should return reasonable policy, albeit random)
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False)
policy = np.random.randint(low=0, high=env.nA, size=env.nS)
value_from_policy = np.random.random(size=env.nS)*10
policy_improvement(env.P, env.nS, env.nA, value_from_policy, policy)

(array([0, 0, 2, 2, 3, 0, 0, 0, 3, 3, 1, 0, 0, 2, 1, 0]), False)

In [7]:
def policy_iteration(P, nS, nA, gamma=0.9, tol=10e-3, max_iter=100):
	"""
	Runs policy iteration.

	You should call the policy_evaluation() and policy_improvement() methods to
	implement this method.

	Parameters
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	tol: float
		tol parameter used in policy_evaluation()
	Returns:
	----------
	value_function: np.ndarray[nS]
	policy: np.ndarray[nS]
	"""

	value_function = np.zeros(nS)
	policy = np.zeros(nS, dtype=int)
 
	for i in range(max_iter):

		value_function = policy_evaluation(P, nS, nA, policy)
		policy, policy_stable = policy_improvement(P, nS, nA, value_function, policy, gamma)

		if policy_stable:
			break
		
		if i==max_iter-1:
			print(f'Policy iteration failed to converge after {max_iter} iterations')

	return value_function, policy

In [8]:
def value_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
	"""
	Learn value function and policy by using value iteration method for a given
	gamma and environment.

	Parameters:
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	tol: float
		Terminate value iteration when
			max |value_function(s) - prev_value_function(s)| < tol
	Returns:
	----------
	value_function: np.ndarray[nS]
	policy: np.ndarray[nS]
	"""

	value_function = np.zeros(nS)
	policy = np.zeros(nS, dtype=int)
	############################
	# YOUR IMPLEMENTATION HERE #


	############################
	return value_function, policy


### Playing the Game

In [14]:
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False) # what happens if we change is_slippery to true?

value_function, policy = policy_iteration(env.P, env.nS, env.nA) # optimal value function and policy

print('Optimal policy: \n', policy.reshape(4,4), '\n')
print('Optimal value function: \n', value_function.reshape(4,4))

Optimal policy: 
 [[1 2 1 0]
 [1 0 1 0]
 [2 1 1 0]
 [0 2 2 0]] 

Optimal value function: 
 [[0.59049 0.6561  0.729   0.6561 ]
 [0.6561  0.      0.81    0.     ]
 [0.729   0.81    0.9     0.     ]
 [0.      0.9     1.      0.     ]]


In [10]:
def play_game(env, policy, max_steps=100):
    """
    Renders policy once on environment. Watch your agent play!

    Parameters
    ----------
    env: gym.core.Environment
        Environment to play on. Must have nS, nA, and P as
        attributes.
    Policy: np.array of shape [env.nS]
        The action to take at a given state
    """
    
    episode_reward = 0
    state = env.reset()
    for t in range(max_steps):
        env.render()
        sleep(0.2)
        action = policy[state]
        state, reward, done, _ = env.step(action)
        episode_reward += reward
        if done:
            break
        clear_output(wait=True)
    env.render()
    if done:
        print(f'Episode reward: {episode_reward}')
    else:
        print(f'Agent did not reach a terminal state in {max_steps} steps')

In [16]:
play_game(env, policy)

  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
Episode reward: 1.0


Now let's try another game with a larger state space, [taxi](https://www.gymlibrary.dev/environments/toy_text/taxi/).

In [17]:
env = gym.make('Taxi-v3')

value_function, policy = policy_iteration(env.P, env.nS, env.nA) # optimal value function and policy

print('Optimal policy: \n', policy, '\n')
print('Optimal value function: \n', value_function)

Optimal policy: 
 [4 4 4 4 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 3
 0 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 0 4 4 4 4 0 0 0 0 0 0 0 0 0 5 0 0 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 2 2 2 2
 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 1
 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 2 2 2 2 0 0 0 0 2 2 2 2 1 2 0 2 1 1
 1 1 2 2 2 2 3 3 3 3 2 2 2 2 1 2 3 2 3 3 3 3 2 2 2 2 3 3 3 3 2 2 2 2 3 2 3
 2 3 3 3 3 2 2 2 2 3 3 3 3 0 0 0 0 3 2 3 0 3 3 3 3 1 1 1 1 3 3 3 3 0 0 0 0
 3 1 3 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 2 2 2 2 1 1 1 1 2
 2 2 2 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 1 1
 1 1 0 0 0 0 1 2 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1
 1 4 4 4 4 1 1 1 1 1 1 5 1 1 1 1 1 2 2 2 2 1 1 1 1 2 2 2 2 1 2 1 2 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 1 1 1 1 4 4 4 4 1 2 1 5 1
 1 1 1 

In [20]:
play_game(env, policy)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Episode reward: 9
